In [34]:
# Rules: https://www.sudokuwiki.org/Sudoku.htm
import string


In [35]:
#init board
sudoku_board = [
    [5, 3, 0, 0, 7, 0, 0, 0, 0],
    [6, 0, 0, 1, 9, 5, 0, 0, 0],
    [0, 9, 8, 0, 0, 0, 0, 6, 0],
    [8, 0, 0, 0, 6, 0, 0, 0, 3],
    [4, 0, 0, 8, 0, 3, 0, 0, 1],
    [7, 0, 0, 0, 2, 0, 0, 0, 6],
    [0, 6, 0, 0, 0, 0, 2, 8, 0],
    [0, 0, 0, 4, 1, 9, 0, 0, 5],
    [0, 0, 0, 0, 8, 0, 0, 7, 9]
]

sudoku_single = [
    [2, 0, 0, 0, 7, 0, 0, 3, 8],
    [0, 0, 0, 0, 0, 6, 0, 7, 0],
    [3, 0, 0, 0, 4, 0, 6, 0, 0],
    [0, 0, 8, 0, 2, 0, 7, 0, 0],
    [1, 0, 0, 0, 0, 0, 0, 0, 6],
    [0, 0, 7, 0, 3, 0, 4, 0, 0],
    [0, 0, 4, 0, 8, 0, 0, 0, 9],
    [0, 6, 0, 4, 0, 0, 0, 0, 0],
    [9, 1, 0, 0, 6, 0, 0, 0, 2]
]

sudoku_nakedpair_row = [
    [4, 0, 0, 0, 0, 0, 9, 3, 8],
    [0, 3, 2, 0, 9, 4, 1, 0, 0],
    [0, 9, 5, 3, 0, 0, 2, 4, 0],
    [3, 7, 0, 6, 0, 9, 0, 0, 4],
    [5, 2, 9, 0, 0, 1, 6, 7, 3],
    [6, 0, 4, 7, 0, 3, 0, 9, 0],
    [9, 5, 7, 0, 0, 8, 3, 0, 0],
    [0, 0, 3, 9, 0, 0, 4, 0, 0],
    [2, 4, 0, 0, 3, 0, 7, 0, 9]
]

sudoku_nakedtriple_row = [
    [0, 7, 0, 4, 0, 8, 0, 2, 9],
    [0, 0, 2, 0, 0, 0, 0, 0, 4],
    [8, 5, 4, 0, 2, 0, 0, 0, 7],
    [0, 0, 8, 3, 7, 4, 2, 0, 0],
    [0, 2, 0, 0, 0, 0, 0, 0, 0],
    [0, 0, 3, 2, 6, 1, 7, 0, 0],
    [0, 0, 0, 0, 9, 3, 6, 1, 2],
    [2, 0, 0, 0, 0, 0, 4, 0, 3],
    [1, 3, 0, 6, 4, 2, 0, 7, 0]
]

In [36]:
class Cell:
    __value: int
    candidates: set = set()
    row: int
    col: int

    def __init__(self, init_value: int, row: int, col: int):
        self.__value = init_value
        self.row = row
        self.col = col
        if init_value == 0:
            self.candidates = {value for value in range(1,10)}

    def get_value(self) -> int:
        return self.__value
    def set_value(self, value):
        self.__value = value
        self.candidates.clear()

    def is_empty(self) -> bool:
        return self.__value == 0
    
    def get_coordinates(self) -> str:
        return string.ascii_uppercase[self.row] + str(self.col+1)


In [37]:
class Board:
    board_values_row: list[list[Cell]] = [] # board values by row
    board_values_col: list[list[Cell]] = [] # board values by col
    board_values_box: list[list[Cell]] = [] # board values by box 1..9

    def __init__(self, sudoku_board: list[list[int]]):
        for row in range(9):
            rows = [Cell(sudoku_board[row][col], row, col) for col in range(9)]
            self.board_values_row.append(rows)

        self.board_values_col = [list(x) for x in zip(*self.board_values_row)]
        
        #box
        # 1 2 3
        # 4 5 6
        # 7 8 9
        for start_row in range(0,9,3):
            for start_col in range(0,9,3):
                result = [Cell]
                for row in range(start_row, start_row+3):
                    for col in range(start_col, start_col+3):
                        result.append(self.board_values_row[row][col])
                self.board_values_box.append(result)

       
    def row_contains_value(self, row: int, value: int) -> bool:
        return value in [x.get_value() for x in self.board_values_row[row]]
    
    def column_contains_value(self, col: int, value: int) -> bool:
        return value in [x.get_value() for x in self.board_values_col[col]]
    
    def box_contains_value(self, row: int, col: int, value: int) -> bool:
        box_row = row // 3
        box_col = col // 3
        start_row = box_row*3
        start_col = box_col*3
        for row in range(start_row, start_row+3):
            for col in range(start_col, start_col+3):
                if self.board_values_row[row][col].get_value() == value:
                    return True
        return False


In [38]:
def display_sudoku(board: Board):
    myboard = board.board_values_row
    print('   ', end='')
    for col in range(9):
        if col % 3 == 0 and col != 0:
            print(" ", end=" ")
        print(col+1, end=" ")
    print('\n')
    
    for row in range(9):       
        if row % 3 == 0 and row != 0:
            print("   " + "-" * 21)  # Prints a line after every 3 rows
        print(string.ascii_uppercase[row], end='  ')

        for col in range(9):
            if col % 3 == 0 and col != 0:
                print("|", end=" ")  # Prints a separator after every 3 columns

            if col == 8:
                print(myboard[row][col].get_value())  # Print the last element and go to the next line
            else:
                print(myboard[row][col].get_value(), end=" ")
    print('\n')


In [39]:
def rule_hidden(board: Board) -> tuple[bool, int, int, int, str]:
    myboard = board.board_values_row

    #unique candidate in row
    for row in range(9):
        for value in range(1,10):
            first_col, occurences = -1, 0
            for col in range(9):
                cell = myboard[row][col]
                if cell.is_empty() and value in cell.candidates:
                    first_col = col
                    occurences += 1
            if occurences == 1:
                message = f'SINGLE: {string.ascii_uppercase[row]}{first_col+1} set to {value}, unique in Row\n'
                return True, value, row, first_col, message
            
    #unique candidate in col
    for col in range(9):
        for value in range(1,10):
            first_row, occurences = -1, 0
            for row in range(9):
                cell = myboard[row][col]
                if cell.is_empty() and value in cell.candidates:
                    first_row = row
                    occurences += 1
            if occurences == 1:
                message = f'SINGLE: {string.ascii_uppercase[first_row]}{col+1} set to {value}, unique in Col\n'
                return True, value, first_row, col, message
            
    #unique candidate in box
    for row_box in range(0,9,3): #start of the box
        for col_box in range(0,9,3): #start of the box
            for value in range(1,10):
                first_row, first_col, occurences = -1, -1, 0
                for row in range(row_box, row_box+3):
                    for col in range(col_box, col_box+3):
                        if cell.is_empty() and value in cell.candidates:
                            first_row = row
                            first_col = col
                            occurences += 1
                if occurences == 1:
                    message = f'SINGLE: {string.ascii_uppercase[first_row]}{first_col+1} set to {value}, unique in Box\n'
                    return True, value, first_row, first_col, message
    return False, 0, 0, 0, ''


    

In [40]:
def naked_pair(row: list[Cell]) -> tuple[bool, str]:
    for index1 in range(8):
        cell1 = row[index1]
        if len(cell1.candidates) == 2:
            for index2 in range(index1+1, 9):
                cell2 = row[index2]
                if cell1.candidates == cell2.candidates: #naked pair found
                    candidates_sorted = sorted(cell1.candidates)
                    message = f'NAKED PAIR: {cell1.get_coordinates()} and {cell2.get_coordinates()}, remove values {candidates_sorted[0]} and {candidates_sorted[1]}'
                    for cell in row: #clean candidates in row
                        if cell != cell1 and cell != cell2:
                            cell.candidates.difference_update(cell1.candidates)

                    return True, message
    return False, ''
                
def rule_nakedpair(board: Board) -> tuple[bool, str]:
    for row in board.board_values_row:
        found, message = naked_pair(row)
        if found:
            return found, message
    
    for col in board.board_values_col:
        found, message = naked_pair(col)
        if found:
            return found, message
        
    for box in board.board_values_box:
        found, message = naked_pair(box)
        if found:
            return found, message

    return False, ''
    

In [41]:
def naked_triple(row: list[Cell]) -> tuple[bool, str]:
    # select cells with 2 or 3 candidates
    cells = [cell for cell in row if len(cell.candidates) == 2 or len(cell.candidates) == 3]
    if len(cells) >= 3:
        for index1 in range(len(cells)-2):
            for index2 in range(index1+1, len(cells)-1):
                for index3 in range(index2+1, len(cells)):
                    cell1, cell2, cell3 = cells[index1], cells[index2], cells[index3]
                    candidates = cell1.candidates | cell2.candidates | cell3.candidates #union
                    if len(candidates) == 3 and \
                       len(cell1.candidates & cell2.candidates) > 0 and \
                       len(cell2.candidates & cell3.candidates) > 0 and \
                       len(cell1.candidates & cell3.candidates) > 0:
                        candidates_sorted = sorted(candidates)
                        message = f'NAKED TRIPLE: {cell1.get_coordinates()} and {cell2.get_coordinates()} and {cell3.get_coordinates()}, remove values {candidates_sorted[0]} and {candidates_sorted[1]} and {candidates_sorted[2]}'
                        for cell in row: #clean candidates in row
                            if cell != cell1 and cell != cell2 and cell != cell3:
                                cell.candidates.difference_update(candidates)
                        return True, message
    return False, ''

def rule_nakedtriple(board: Board) -> tuple[bool, str]:
    for row in board.board_values_row:
        found, message = naked_triple(row)
        if found:
            return found, message
    
    for col in board.board_values_col:
        found, message = naked_triple(col)
        if found:
            return found, message
        
    for box in board.board_values_box:
        found, message = naked_triple(box)
        if found:
            return found, message

    return False, ''

In [ ]:
def hidden_pair(row: list[Cell]) -> tuple[bool, str]:
    ct = {i:[] for i in range(1,10)}
    for cell in row:
        for candidate in cell.candidates:
            ct[candidate] += cell
    new_ct = {k:v for k,v in ct.items if len(v)==2}
    if len(new_ct) >= 2:
        values = list(new_ct.keys())
        for i in range(1,9):
            for j in range(i+1, 10):
                for k in range(1,9):
                    for l in range(k+1, 10):
                        cell1, cell2 = row[k], row[l]
                        if i in values and j in values and i in cell1.candidates and i in cell2.candidates and j in cell1.candidates and j in cell2.candidates:
                            message = f'NAKED PAIR: {row[k].get_coordinates()} and {row[l].get_coordinates()}, remove values {i} and {j}'
                            for cell in row: #clean candidates in row
                                if cell != cell1 and cell != cell2:
                                    cell.candidates.difference_update({i,j})
                            return True, message
    return False, ''


def rule_hidden_pair(board: Board) -> tuple[bool, str]:
    for row in board.board_values_row:
        found, message = hidden_pair(row)
        if found:
            return found, message
    
    for col in board.board_values_col:
        found, message = hidden_pair(col)
        if found:
            return found, message
        
    for box in board.board_values_box:
        found, message = hidden_pair(box)
        if found:
            return found, message

    return False, ''

In [42]:
def rule_clean_candidates(board: Board):
    myboard = board.board_values_row
    for row in range(9):
        for col in range(9):
            cell = myboard[row][col]
            if cell.is_empty():
                candidates_copy = cell.candidates.copy()
                for value in candidates_copy:
                    if board.row_contains_value(row, value) or board.column_contains_value(col, value) or board.box_contains_value(row, col, value):
                        cell.candidates.remove(value)

In [43]:
def display_solve(board: Board, found: bool, value: int, row: int, col: int, message: str):
    if found:
        print(message)
        board.board_values_row[row][col].set_value(value)
        display_sudoku(board)

In [95]:
#Main test
board = Board(sudoku_single.copy())

display_sudoku(board)
found = True
max_level = 0
while(found):
    rule_clean_candidates(board)
    found, value, row, col, message = rule_hidden(board)
    display_solve(board, found, value, row, col, message)
    if found: 
        max_level = max(max_level, 1)
        continue
    found, message = rule_nakedpair(board)
    if found:
        print(message)
    if found: 
        max_level = max(max_level, 2)
print(f'Max_level: {max_level}')
    

   1 2 3   4 5 6   7 8 9 

A  2 0 0 | 0 7 0 | 0 3 8
B  0 0 0 | 0 0 6 | 0 7 0
C  3 0 0 | 0 4 0 | 6 0 0
   ---------------------
D  0 0 8 | 0 2 0 | 7 0 0
E  1 0 0 | 0 0 0 | 0 0 6
F  0 0 7 | 0 3 0 | 4 0 0
   ---------------------
G  0 0 4 | 0 8 0 | 0 0 9
H  0 6 0 | 4 0 0 | 0 0 0
I  9 1 0 | 0 6 0 | 0 0 2


SINGLE: A2 set to 4, unique in Row

   1 2 3   4 5 6   7 8 9 

A  2 4 0 | 0 7 0 | 0 3 8
B  0 0 0 | 0 0 6 | 0 7 0
C  3 0 0 | 0 4 0 | 6 0 0
   ---------------------
D  0 0 8 | 0 2 0 | 7 0 0
E  1 0 0 | 0 0 0 | 0 0 6
F  0 0 7 | 0 3 0 | 4 0 0
   ---------------------
G  0 0 4 | 0 8 0 | 0 0 9
H  0 6 0 | 4 0 0 | 0 0 0
I  9 1 0 | 0 6 0 | 0 0 2


SINGLE: A3 set to 6, unique in Row

   1 2 3   4 5 6   7 8 9 

A  2 4 6 | 0 7 0 | 0 3 8
B  0 0 0 | 0 0 6 | 0 7 0
C  3 0 0 | 0 4 0 | 6 0 0
   ---------------------
D  0 0 8 | 0 2 0 | 7 0 0
E  1 0 0 | 0 0 0 | 0 0 6
F  0 0 7 | 0 3 0 | 4 0 0
   ---------------------
G  0 0 4 | 0 8 0 | 0 0 9
H  0 6 0 | 4 0 0 | 0 0 0
I  9 1 0 | 0 6 0 | 0 0 2


SINGLE: B4 set t

In [11]:
#test nakedpair_row

board = Board(sudoku_nakedpair_row.copy())
rule_clean_candidates(board)
found, message = rule_nakedpair(board)
if not found:
    print('ERROR')
for i in (3,4,5):
    assert len({1,6} & board.board_values_row[0][i].candidates) == 0

print('test success')
print(message)

test success


In [10]:
#test nakedpair

board = Board(sudoku_nakedpair_row.copy())
rule_clean_candidates(board)
sudoku_nakedpair = board.board_values_row[0]
found, message = naked_pair(sudoku_nakedpair)
assert found
for i in (3,4,5):
    assert len({1,6} & sudoku_nakedpair[i].candidates) == 0

print('test success')
print(message)

In [46]:
#test nakedtriple

board = Board(sudoku_nakedtriple_row.copy())
rule_clean_candidates(board)
sudoku_nakedtriple = board.board_values_row[4]
found, message = naked_triple(sudoku_nakedtriple)
assert found
for i in (0,2,6,7):
    assert len({5,8,9} & sudoku_nakedtriple[i].candidates) == 0

print('test success')
print(message)

test success
NAKED TRIPLE: E4 and E5 and E6, remove values 5 and 8 and 9
